In [ ]:
import pandas as pd

In [23]:
df = pd.read_csv('data/salaries_by_college_major.csv')

In [24]:
df.head() # Peeks the top 5 rows from the table

,Undergraduate Major,Starting Median Salary,Mid-Career Median Salary,Mid-Career 10th Percentile Salary,Mid-Career 90th Percentile Salary,Group
0,Accounting,46000.0,77100.0,42200.0,152000.0,Business
1,Aerospace Engineering,57700.0,101000.0,64300.0,161000.0,STEM
2,Agriculture,42600.0,71900.0,36300.0,150000.0,Business
3,Anthropology,36800.0,61500.0,33800.0,138000.0,HASS
4,Architecture,41600.0,76800.0,50600.0,136000.0,Business


In [ ]:
df.shape # Gives the number of rows and column 

In [ ]:
df.columns

In [63]:
# Handling Missing and junk values
'''To check if there are any field with missing values shown as NaN--> Not a number
NanN refers to empty cell or a cell which contains string values instead of number'''
df.isna()
df.tail() # Gives the last 5 rows

# To handle NaN we manually delete or use .dropna() method
# First we create a new dataframe and then project last 5 rows
clean_df = df.dropna()
clean_df.tail()

,Undergraduate Major,Starting Median Salary,Mid-Career Median Salary,Mid-Career 10th Percentile Salary,Mid-Career 90th Percentile Salary,Group
45,Political Science,40800.0,78200.0,41200.0,168000.0,HASS
46,Psychology,35900.0,60400.0,31600.0,127000.0,HASS
47,Religion,34100.0,52000.0,29700.0,96400.0,HASS
48,Sociology,36500.0,58200.0,30700.0,118000.0,HASS
49,Spanish,34000.0,53100.0,31000.0,96400.0,HASS


In [64]:
# Accessing the column and the cell
clean_df['Starting Median Salary'].max()
clean_df['Starting Median Salary'].idxmax() # Gives the row with the largest value
# clean_df['Undergraduate Major'].loc[43]  #  .loc property gives the row corresponding to that column
# Also we can use
clean_df['Undergraduate Major'][43]

'Physician Assistant'

In [45]:
'''What college major has the highest mid-career salary? How much do graduates with this major earn?
 (Mid-career is defined as having 10+ years of experience)'''

clean_df['Mid-Career Median Salary'].max() # 10,7000
clean_df['Mid-Career Median Salary'].idxmax()
var = clean_df['Undergraduate Major'][8]  # Chemical Engineering
highest = clean_df['Starting Median Salary'][8] # 63200

''' Which college major has the lowest mid-career salary and how much can people expect to earn with this degree'''
clean_df['Mid-Career Median Salary'].min() # 52000
ind = clean_df['Mid-Career Median Salary'].idxmin()
clean_df['Undergraduate Major'][ind] # Education

'Education'

In [65]:
'''Which college major has the lowest starting salary and how much do graduates earn after university'''
clean_df['Starting Median Salary'].idxmin()
lowest  = clean_df['Undergraduate Major'][49] # Spanish
clean_df['Starting Median Salary'].min() # 34000

34000.0

In [77]:
'''Sorting Values & Adding Columns: Majors with the Most Potential vs Lowest Risk'''
# To subtract use '-' or .subtract() method
# Subtract columns and insert new column
diff = clean_df['Mid-Career 90th Percentile Salary'] - clean_df['Mid-Career 10th Percentile Salary']
clean_df.insert(1, 'spread', diff)
clean_df.head()
low_risk = clean_df.sort_values('spread')
low_risk[['Undergraduate Major','spread']].head()


,Undergraduate Major,spread
40,Nursing,50700.0
43,Physician Assistant,57600.0
41,Nutrition,65300.0
49,Spanish,65400.0
27,Health Care Administration,66400.0


In [82]:
low_risk = clean_df.sort_values('spread',ascending=False)
low_risk[['Undergraduate Major','spread']].head()

,Undergraduate Major,spread
17,Economics,159400.0
22,Finance,147800.0
37,Math,137800.0
36,Marketing,132900.0
42,Philosophy,132500.0


In [80]:
# Find the top 5 degrees with the highest values in the 90th percentile.
highest_potential = clean_df.sort_values('Mid-Career 90th Percentile Salary',ascending=False)
highest_potential[['Undergraduate Major','Mid-Career 90th Percentile Salary']].head()

,Undergraduate Major,Mid-Career 90th Percentile Salary
17,Economics,210000.0
22,Finance,195000.0
8,Chemical Engineering,194000.0
37,Math,183000.0
44,Physics,178000.0


In [131]:
import numpy as np
'''Grouping and Pivoting Data with Pandas'''
group = clean_df.groupby('Group')
# clean_df = clean_df.drop(columns=['spread'])
clean_df['spread'] = clean_df['Mid-Career 90th Percentile Salary'] - clean_df['Mid-Career 10th Percentile Salary']
clean_df['spread'] = pd.to_numeric(clean_df['spread'], errors='coerce')
numeric_cols = clean_df.select_dtypes(include=[np.number]).columns  # Using Numpy

pd.options.display.float_format = '{:,.2f}'.format 
group[numeric_cols].mean()

,Starting Median Salary,Mid-Career Median Salary,Mid-Career 10th Percentile Salary,Mid-Career 90th Percentile Salary,spread
Group,,,,,
Business,"44,633.33","75,083.33","43,566.67","147,525.00","103,958.33"
HASS,"37,186.36","62,968.18","34,145.45","129,363.64","95,218.18"
STEM,"53,862.50","90,812.50","56,025.00","157,625.00","101,600.00"


In [141]:
# PAYSCALE
import requests
from bs4 import BeautifulSoup

url = 'https://www.payscale.com/college-salary-report/majors-that-pay-you-back/bachelors'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
tables = soup.find_all('table')
if tables:
    df = pd.read_html(str(tables[0]))[0].copy()
    print(df.head())
